In [ ]:
!git clone https://github.com/bart-custers/agentic_actuaries.git
%cd agentic_actuaries

In [ ]:
!pip install -r requirements.txt

In [ ]:
%cd /content/agentic_actuaries
!git pull

In [ ]:
import sys
import os

sys.path.append(r"c:\Users\bart_\Documents\git_repo\agentic_actuaries")

os.chdir("..") # move one level up
#os.chdir("agentic_actuaries")

print(os.getcwd())

In [2]:
from agents.central_hub import CentralHub
from agents.data_prep_agent import DataPrepAgent
from agents.modelling_agent import ModellingAgent
from agents.review_agent import ReviewingAgent
from agents.explanation_agent import ExplanationAgent

In [3]:
from llms.wrappers import LLMWrapper
llm = LLMWrapper(backend="phi3mini")

Loading microsoft/phi-3-mini-128k-instruct ... (this may take 1–2 minutes the first time)


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: [enforce fail at alloc_cpu.cpp:121] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3221225472 bytes.

In [ ]:
# Initialize LLaMA LLM
# from llms.llama_setup import get_llama7b_llm
# llm = get_llama7b_llm()

In [ ]:
# Initialize hub and agents
hub = CentralHub()

#data_prep = DataPrepAgent("Data Prep Agent", hub)
data_prep = DataPrepAgent("Data Prep", hub, llm=llm)
modelling = ModellingAgent("Modelling Agent", hub)
reviewing = ReviewingAgent("Reviewing Agent", hub)
explanation = ExplanationAgent("Explanation Agent", hub)

In [ ]:
raw_path = "data/raw/freMTPL2freq.csv"

print("\n🚀 Step 1: LLM-based data cleaning and refinement...")
prep_result = hub.send("Hub", data_prep, {
    "task": "clean_and_refine",
    "data_path": raw_path
})

print("\n=== DataPrep (LLM) Result ===")
print(json.dumps(prep_result, indent=2))

# Inspect hub memory for pipeline metadata
print("\nHub memory keys after DataPrep:", list(hub.memory.keys()))

In [ ]:
# Send a test cleaning task
test_message = {"task": "clean_data", "data_path": "data/raw/freMTPL2freq.csv"}

# Send through hub
output = hub.send("TestNotebook", data_prep_agent, test_message)

print("Data Prep Agent Output:")
print(output)

# Optional: send to deterministic agent
mod_output = hub.send("TestNotebook", modelling_agent, {"task": "train_glm"})
print("\nModelling Agent Output:")
print(mod_output)

# Inspect hub memory/logs
hub.show_memory()


In [ ]:
# --- run full flow ---
hub.send("Hub", data_prep, {"task": "clean_data", "data_path": "data/raw/freMTPL2freq.csv"})
hub.send("Hub", modelling, {"task": "train_glm"})
hub.send("Hub", reviewing, {"task": "review_model"})
hub.send("Hub", explanation, {"task": "explain_model"})

In [ ]:
hub.show_memory()

In [ ]:
print("\nAudit metrics:")
print(hub.memory["audit_report"]["r2_score"])